In [1]:
curr_dir = '/home/vayzenbe/GitHub_Repos/GiNN'

import sys
sys.path.insert(1, f'{curr_dir}/Models')
import os, argparse
import torch
import torch.nn as nn
from torchvision import transforms
from torch.autograd import Variable
from PIL import Image, ImageOps,  ImageFilter
from itertools import chain
import pandas as pd
import numpy as np
import cornet
import model_funcs
import matplotlib.pyplot as plt
from statistics import mean
%matplotlib inline
import LoadFrames
import load_without_faces

In [2]:
stim_dir = f"{curr_dir}/Stim/"
weights_dir = f"/lab_data/behrmannlab/vlad/ginn/model_weights"

im_cond = ['cropped_face', 'schematic']


train_cond = [ 'imagenet_oneface','mixed_imagenet_vggface']
n_classes = [601,1200]


#train_cond = ['general']
#n_classes = [600]
#train_cond = ['imagenet_objects']
#model_types = ['vggface']
model_epochs = [0, 1, 5, 10, 15, 20, 25, 30]


model_type = 'classify'
layer =['out', 'aIT','pIT', 'V4', 'V2', 'V1']



transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.Grayscale(num_output_channels=3),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                             std=[0.229, 0.224, 0.225])])

cos = nn.CosineSimilarity(dim=0, eps=1e-6)

In [3]:
def load_files(dirName):
    listOfFiles = list()
    for (dirpath, dirnames, filenames) in os.walk(dirName):
        listOfFiles += [os.path.join(dirpath, file) for file in filenames]
            
    return listOfFiles    

In [4]:
def extract_acts(model):
    """
    Extracts the activations for a series of images
    """
    model.eval()

    with torch.no_grad():
        for im, label in loader:
            im = im.cuda()
            output = model(im)
            output =output.view(output.size(0), -1)
            
    return output, label

## Viewpoint invariance experiment

In [5]:
def determine_ident_orient(file_name):
    """
    Takes a file name and extracts the identity and head orientation
    """

    #file_name = file_name.replace(f'{stim_dir}/{cond_dir}/','')
    identity = file_name.replace('_left.jpg', '')
    identity = identity.replace('_right.jpg', '')
    identity = identity.replace('_front.jpg', '')

    orient = file_name.replace(f'{identity}_', '')
    orient = orient.replace('.jpg', '')
    
    return identity, orient


In [6]:
def determine_conds(hab_ident, test_ident, hab_orient, test_orient):
    """
    determines the identity and orientation condition from two images
    """
    
    if hab_ident == test_ident:
        ident_cond = "same"
    else:
        ident_cond = "diff"

    if hab_orient == test_orient:
        orient_cond = "same"
    else:
        orient_cond = "diff"

    return ident_cond, orient_cond

In [7]:
test_cond = 'cfmt'
im_dir = f'{stim_dir}/{test_cond}'
im_files = load_files(im_dir)
im_count = len(im_files)

#set up data loader for full sample
dataset = load_without_faces.load_stim(train_dir, exclude_im, exclude_folder, transform=transform)
loader = torch.utils.data.DataLoader(dataset, batch_size=im_count, shuffle=False,num_workers = 4, pin_memory=True)

for tcn, tc  in enumerate(train_cond): #loop through model conditions
    for ee in model_epochs: #loop through model epochs
        #load model 
        base_model = model_funcs.load_model(model_type, tc, ee, weights_dir, n_classes[tcn])
        
        for ll in layer:
            model = model_funcs.remove_layer(base_model, ll)
            output, label = extract_acts(model)


            df = pd.DataFrame(columns = ['model_type', 'train_type', 'epoch',
                                                 'im1', 'im2', 'identity', 'orient',
                                                 'similarity']) 
            for ii in range(0,len(output)):
                im1_ident, im1_orient = determine_ident_orient(label[ii])
                for kk in range(ii+1,len(output)):
                    im2_ident, im2_orient = determine_ident_orient(label[kk])

                    sim = cos(output[ii], output[kk])
                    sim =sim.cpu().numpy()

                    ident_cond, orient_cond = determine_conds(im1_ident, im2_ident, im1_orient, im2_orient)

                    trial_results = [model_type, tc, ee, label[ii], label[kk],ident_cond, orient_cond,sim]
                    #print(trial_results)
                    trial_results = pd.Series(trial_results, index = df.columns)
                    df = df.append(trial_results, ignore_index=True)


            df.to_csv(f'{curr_dir}/Results/orientation/{model_type}_{tc}_{ee}_{ll}.csv', sep =",", index = False)



In [8]:
def image_loader(image_name):
    im = Image.open(image_name).convert("RGB")
    #make image grayscale
    image_name = ImageOps.grayscale(im).convert("RGB")
    #image_name = image_name.filter(ImageFilter.GaussianBlur(radius=4))
    """load image, returns cuda tensor"""
    im = Variable(normalize(to_tensor(scaler(im))).unsqueeze(0))
    im = im.cuda()
    return im

In [9]:
def calc_acc(df, orient):
    #Calc front acc
    acc =[]
    same = df['test_loss'][(df["identity"] == 'same') & (df["orient"] == orient)]
    diff = df['test_loss'][(df["identity"] == 'diff') & (df["orient"] == orient)]

    for ss in same:
        for dd in diff:
            acc.append(dd/(ss+dd))   
    
    return pd.Series(acc).mean(), pd.Series(acc).sem()


In [10]:
def plot_line(df_plot, df_conds):
    
    plt.figure()
    for dc in df_conds:    
        plt.errorbar(df_plot['epoch'], df_plot[dc],  yerr=df_plot[f'{dc}_se'], label=dc)
    
  
    plt.xlabel("Epochs")
    plt.ylabel("Activation/Preference")
    #ax.set_ylabel('Activation')
    #print(f'{df_plot["model][0]}-test')
    plt.title(f'{df_plot["model_type"][0]} model trained with {df_plot["train_type"][0]}')
    plt.legend(bbox_to_anchor=(1.05, 1.0), loc='upper left')
    plt.tight_layout()
    plt.savefig(f'{curr_dir}/Results/orientation/figures/{df_plot["model_type"][0]}_{df_plot["train_type"][0]}.png',bbox_inches='tight')
    

## Other species experiment

In [11]:
def calc_species_acc(df):
    #Calc front acc
    acc =[]
    same = df['test_loss'][(df["identity"] == 'same')]
    diff = df['test_loss'][(df["identity"] == 'diff')]

    for ss in same:
        for dd in diff:
            acc.append(dd/(ss+dd))   
    
    return pd.Series(acc).mean(), pd.Series(acc).sem()


In [12]:
test_dirs = ['facegen_front','monkey_front']
for test_cond in test_dirs:
    
    im_dir = f'{stim_dir}/{test_cond}'
    im_files = load_files(im_dir)
    im_count = len(im_files)

    #set up data loader for full sample
    dataset = LoadFrames.LoadFrames(im_dir,  transform=transform)
    loader = torch.utils.data.DataLoader(dataset, batch_size=im_count, shuffle=False,num_workers = 4, pin_memory=True)

    for tcn, tc  in enumerate(train_cond):
        for ee in model_epochs:
            base_model = model_funcs.load_model(model_type, tc, ee, weights_dir, n_classes[tcn])
            
            for ll in layer:
            #load model 
                model = model_funcs.remove_layer(base_model, ll)

                output, label = extract_acts(model)

                df = pd.DataFrame(columns = ['model_type', 'train_type', 'epoch',
                                                     'im1', 'im2', 'similarity']) 
                for ii in range(0,len(output)):

                    for kk in range(ii+1,len(output)):

                        sim = cos(output[ii], output[kk])
                        sim =sim.cpu().numpy()

                        trial_results = [model_type, tc, ee, label[ii], label[kk],sim]
                        #print(trial_results)
                        trial_results = pd.Series(trial_results, index = df.columns)
                        df = df.append(trial_results, ignore_index=True)


                df.to_csv(f'{curr_dir}/Results/other_species/{model_type}_{tc}_{ee}_{ll}_{test_cond}.csv', sep =",", index = False)